<a href="https://colab.research.google.com/github/Ismat-Samadov/colab_notebooks/blob/main/digital_photo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import cv2                  # OpenCV library for computer vision tasks
import numpy as np          # NumPy for numerical operations and array handling
import random              # For generating random numbers (used in noise generation)
import os                  # For file path operations and checking if files exist

"""
1. Edge Detection with Sobel Operator
-----------------------------------
The Sobel operator calculates the gradient of the image intensity,
highlighting regions of high spatial frequency (edges).
"""
def sobel_edge_detection(image_path):
    # Read image from the specified path
    img = cv2.imread(image_path)  # Load the image in BGR format (OpenCV default)

    # Check if image was loaded successfully
    if img is None:
        print(f"Error: Could not read image from {image_path}")  # Print error message
        return None, None  # Return None values to indicate failure

    # Convert the color image to grayscale (required for many image processing operations)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert from BGR to grayscale

    # Apply Sobel operator in X direction (horizontal edges)
    # cv2.CV_64F specifies 64-bit float output, 1,0 means X direction only, ksize=3 is the kernel size
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)

    # Apply Sobel operator in Y direction (vertical edges)
    # cv2.CV_64F specifies 64-bit float output, 0,1 means Y direction only, ksize=3 is the kernel size
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    # Calculate the magnitude of gradients (combines X and Y gradients)
    # This gives us the edge strength at each pixel
    edges = cv2.magnitude(sobelx, sobely)

    # Convert back to uint8 format (0-255 range) for saving and display
    # The convertScaleAbs function takes absolute values and scales appropriately
    edges = cv2.convertScaleAbs(edges)

    # Save the edge-detected image to file
    cv2.imwrite("sobel_edges.jpg", edges)

    # Return both the grayscale image and the edge map for further processing
    return gray, edges

"""
2. Histogram Equalization
-----------------------
Enhances contrast by redistributing intensity values.
Useful for images with poor contrast.
"""
def histogram_equalization(gray_img):
    # Check if the input image is valid
    if gray_img is None:
        print("Error: Cannot perform histogram equalization on None image")  # Print error message
        return None  # Return None to indicate failure

    # Apply histogram equalization
    # This redistributes pixel intensities to improve contrast
    # Works only on grayscale images (8-bit, single channel)
    equalized = cv2.equalizeHist(gray_img)

    # Save the equalized image to file
    cv2.imwrite("hist_equalized.jpg", equalized)

    # Return the equalized image for further processing
    return equalized

"""
3. Image Smoothing Comparison
---------------------------
Gaussian Filter: Uses Gaussian function for smoothing (better preserves edges)
Median Filter: Replaces each pixel with median of neighboring pixels (good for salt-and-pepper noise)
"""
def compare_smoothing_filters(gray_img):
    # Check if the input image is valid
    if gray_img is None:
        print("Error: Cannot apply smoothing filters on None image")  # Print error message
        return None, None  # Return None values to indicate failure

    # Apply Gaussian blur
    # (5,5) is the kernel size (width and height)
    # 0 is the standard deviation in X direction, 0 means auto-calculated
    gauss_blur = cv2.GaussianBlur(gray_img, (5, 5), 0)

    # Apply Median blur
    # 5 is the kernel size (must be odd)
    # Replaces each pixel with the median value in the neighborhood
    median_blur = cv2.medianBlur(gray_img, 5)

    # Stack the results horizontally for side-by-side comparison
    # np.hstack concatenates arrays horizontally (column-wise)
    combined = np.hstack((gauss_blur, median_blur))

    # Save the combined image to file
    cv2.imwrite("blur_comparison.jpg", combined)

    # Return both blurred images for further processing
    return gauss_blur, median_blur

"""
4. Color Channel Separation
------------------------
Splits an RGB image into its component channels.
OpenCV uses BGR format by default.
"""
def split_color_channels(image_path):
    # Read image from the specified path
    img = cv2.imread(image_path)  # Load the image in BGR format (OpenCV default)

    # Check if image was loaded successfully
    if img is None:
        print(f"Error: Could not read image from {image_path}")  # Print error message
        return None, (None, None, None)  # Return None values to indicate failure

    # Split the image into its three color channels (Blue, Green, Red)
    # OpenCV stores images in BGR order, not RGB
    b, g, r = cv2.split(img)

    # Save each channel as a separate grayscale image
    cv2.imwrite("blue_channel.jpg", b)  # Save blue channel
    cv2.imwrite("green_channel.jpg", g)  # Save green channel
    cv2.imwrite("red_channel.jpg", r)  # Save red channel

    # Return the original image and a tuple containing individual channels
    return img, (b, g, r)

"""
5. Color Space Conversion and Thresholding
---------------------------------------
HSV (Hue, Saturation, Value) is better for color-based segmentation
than RGB. This isolates blue objects in the image.
"""
def isolate_color(img):
    # Check if the input image is valid
    if img is None:
        print("Error: Cannot isolate color from None image")  # Print error message
        return None  # Return None to indicate failure

    # Convert from BGR to HSV color space
    # HSV separates color (hue) from intensity (value) and purity (saturation)
    # Making it better for color-based segmentation
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define lower and upper bounds for blue color in HSV
    # Hue range for blue is approximately 100-140 (out of 180 in OpenCV)
    # High saturation (150-255) ensures we get vivid blues
    # Value range (0-255) covers all brightness levels
    lower_blue = np.array([100, 150, 0])  # Lower bound for blue color
    upper_blue = np.array([140, 255, 255])  # Upper bound for blue color

    # Create a binary mask where blue pixels are white (255) and others are black (0)
    # inRange checks if each pixel in hsv falls between lower_blue and upper_blue
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Apply the mask to the original image
    # bitwise_and keeps only the pixels where mask has white pixels (255)
    result = cv2.bitwise_and(img, img, mask=mask)

    # Save the result to file
    cv2.imwrite("blue_isolated.jpg", result)

    # Return the color-isolated image
    return result

"""
6. Image Transformations
---------------------
Resize and rotate the image using affine transformations.
"""
def transform_image(img):
    # Check if the input image is valid
    if img is None:
        print("Error: Cannot transform None image")  # Print error message
        return None, None  # Return None values to indicate failure

    # Get image dimensions (height, width)
    # The shape attribute returns (height, width, channels)
    # We only need the first two elements (height and width)
    (h, w) = img.shape[:2]

    # Resize the image to half its original size
    # w//2 and h//2 are the new width and height (integer division)
    resized = cv2.resize(img, (w//2, h//2))

    # Save the resized image to file
    cv2.imwrite("resized.jpg", resized)

    # Calculate the center of the original image
    # This will be the pivot point for rotation
    center = (w // 2, h // 2)

    # Create a rotation matrix
    # center is the pivot point, 45 is the angle in degrees, 1.0 is the scale factor
    M = cv2.getRotationMatrix2D(center, 45, 1.0)

    # Apply the rotation to the image using the rotation matrix
    # (w, h) specifies the size of the output image
    rotated = cv2.warpAffine(img, M, (w, h))

    # Save the rotated image to file
    cv2.imwrite("rotated.jpg", rotated)

    # Return both the resized and rotated images
    return resized, rotated

"""
7. Noise Addition and Removal
--------------------------
Adds salt-and-pepper noise and removes it with median filtering.
"""
def add_and_remove_noise(image_path):
    # Read image from the specified path as grayscale
    # The 0 flag loads the image directly as grayscale
    img = cv2.imread(image_path, 0)

    # Check if image was loaded successfully
    if img is None:
        print(f"Error: Could not read image from {image_path}")  # Print error message
        return None, None  # Return None values to indicate failure

    # Create a copy of the image to add noise to
    # This preserves the original image
    noisy = img.copy()

    # Get image dimensions
    rows, cols = noisy.shape

    # Add salt noise (white pixels)
    # Loop 500 times to add 500 white pixels at random locations
    for i in range(500):
        # Generate random coordinates within image dimensions
        x = random.randint(0, rows - 1)  # Random x-coordinate
        y = random.randint(0, cols - 1)  # Random y-coordinate
        # Set the pixel at (x,y) to white (255)
        noisy[x, y] = 255

    # Add pepper noise (black pixels)
    # Loop 500 times to add 500 black pixels at random locations
    for i in range(500):
        # Generate random coordinates within image dimensions
        x = random.randint(0, rows - 1)  # Random x-coordinate
        y = random.randint(0, cols - 1)  # Random y-coordinate
        # Set the pixel at (x,y) to black (0)
        noisy[x, y] = 0

    # Apply median filter to remove salt-and-pepper noise
    # 3 is the kernel size (must be odd)
    # Median filter replaces each pixel with the median value in the neighborhood
    filtered = cv2.medianBlur(noisy, 3)

    # Save both the noisy and filtered images
    cv2.imwrite("noisy_image.jpg", noisy)
    cv2.imwrite("denoised_image.jpg", filtered)

    # Return both the noisy and filtered images
    return noisy, filtered

"""
8. Edge Detection Comparison
-------------------------
Compares Sobel edges with Canny edge detection.
Canny is more sophisticated and includes:
- Noise reduction
- Gradient calculation
- Non-maximum suppression
- Hysteresis thresholding
"""
def compare_edge_detectors(img):
    # Check if the input image is valid
    if img is None:
        print("Error: Cannot compare edge detectors on None image")  # Print error message
        return None, None  # Return None values to indicate failure

    # Apply Sobel operator in both X and Y directions
    # cv2.CV_64F specifies 64-bit float output, 1,1 means both X and Y directions
    sobel = cv2.Sobel(img, cv2.CV_64F, 1, 1, ksize=3)

    # Convert back to uint8 format (0-255 range)
    sobel = cv2.convertScaleAbs(sobel)

    # Apply Canny edge detector
    # 100 is the lower threshold, 200 is the upper threshold
    # Pixels with gradient magnitude > upper threshold are edges
    # Pixels connected to edges with gradient magnitude > lower threshold are also edges
    canny = cv2.Canny(img, 100, 200)

    # Stack the results horizontally for side-by-side comparison
    comparison = np.hstack((sobel, canny))

    # Save the comparison image to file
    cv2.imwrite("sobel_vs_canny.jpg", comparison)

    # Return both edge detection results
    return sobel, canny

"""
9. Morphological Operations
-----------------------
Operations that process images based on shapes.
- Dilation: Expands shapes, fills gaps
- Erosion: Shrinks shapes, removes small objects
"""
def apply_morphological_ops(img):
    # Check if the input image is valid
    if img is None:
        print("Error: Cannot apply morphological operations on None image")  # Print error message
        return None, None, None  # Return None values to indicate failure

    # Create binary image using thresholding
    # If pixel value > 127, set to 255 (white), otherwise set to 0 (black)
    # The underscore (_) is used to ignore the return value of threshold (the threshold used)
    _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # Define a 3x3 kernel of ones
    # This kernel determines the neighborhood used in morphological operations
    kernel = np.ones((3, 3), np.uint8)

    # Apply dilation
    # Expands white regions (foreground)
    # iterations=1 means apply once
    dilated = cv2.dilate(binary, kernel, iterations=1)

    # Apply erosion
    # Shrinks white regions (foreground)
    # iterations=1 means apply once
    eroded = cv2.erode(binary, kernel, iterations=1)

    # Stack the results horizontally for side-by-side comparison
    combined = np.hstack((binary, dilated, eroded))

    # Save the comparison image to file
    cv2.imwrite("morph_operations.jpg", combined)

    # Return all three images
    return binary, dilated, eroded

"""
Main function to run all operations
"""
def process_images(image1_path='image1.jpg', image2_path='image2.jpg', photo_path='photo.jpg'):
    # Initialize counters for success tracking
    success_count = 0  # Count of successful operations
    total_operations = 9  # Total number of operations to perform

    # Check if input files exist and print warnings if they don't
    if not os.path.exists(image1_path):
        print(f"Warning: {image1_path} not found.")  # Print warning for missing file
    if not os.path.exists(image2_path):
        print(f"Warning: {image2_path} not found.")  # Print warning for missing file
    if not os.path.exists(photo_path):
        print(f"Warning: {photo_path} not found.")  # Print warning for missing file

    # 1. Sobel Edge Detection
    print("Performing Sobel edge detection...")  # Status update
    # Call the function and store its return values
    gray, edges = sobel_edge_detection(image1_path)
    # Check if the operation was successful
    if gray is not None and edges is not None:
        success_count += 1  # Increment success counter

        # 2. Histogram Equalization
        print("Performing histogram equalization...")  # Status update
        # Call the function and store its return value
        equalized = histogram_equalization(gray)
        # Check if the operation was successful
        if equalized is not None:
            success_count += 1  # Increment success counter

        # 3. Image Smoothing
        print("Comparing smoothing filters...")  # Status update
        # Call the function and store its return values
        gauss_blur, median_blur = compare_smoothing_filters(gray)
        # Check if the operation was successful
        if gauss_blur is not None and median_blur is not None:
            success_count += 1  # Increment success counter

        # 8. Edge Detection Comparison
        print("Comparing edge detectors...")  # Status update
        # Call the function and store its return values
        sobel, canny = compare_edge_detectors(gray)
        # Check if the operation was successful
        if sobel is not None and canny is not None:
            success_count += 1  # Increment success counter

        # 9. Morphological Operations
        print("Applying morphological operations...")  # Status update
        # Call the function and store its return values
        binary, dilated, eroded = apply_morphological_ops(gray)
        # Check if the operation was successful
        if binary is not None and dilated is not None and eroded is not None:
            success_count += 1  # Increment success counter

    # 4. Color Channel Separation
    print("Separating color channels...")  # Status update
    # Call the function and store its return values
    img_color, (b, g, r) = split_color_channels(photo_path)
    # Check if the operation was successful
    if img_color is not None and b is not None and g is not None and r is not None:
        success_count += 1  # Increment success counter

        # 5. HSV Color Isolation
        print("Isolating colors in HSV space...")  # Status update
        # Call the function and store its return value
        blue_isolated = isolate_color(img_color)
        # Check if the operation was successful
        if blue_isolated is not None:
            success_count += 1  # Increment success counter

        # 6. Image Transformations
        print("Applying geometric transformations...")  # Status update
        # Call the function and store its return values
        resized, rotated = transform_image(img_color)
        # Check if the operation was successful
        if resized is not None and rotated is not None:
            success_count += 1  # Increment success counter

    # 7. Noise Addition and Removal
    print("Adding and removing noise...")  # Status update
    # Call the function and store its return values
    noisy, filtered = add_and_remove_noise(image2_path)
    # Check if the operation was successful
    if noisy is not None and filtered is not None:
        success_count += 1  # Increment success counter

    # Print summary of operations
    print(f"\nCompleted {success_count} out of {total_operations} operations successfully.")
    # If all operations were successful, print a success message
    if success_count == total_operations:
        print("All image processing operations completed successfully.")
    # Inform where output files are saved
    print("Output images saved to current directory.")

# Script entry point
if __name__ == "__main__":
    # Call the main function with default image paths
    # You can specify your image paths here by changing the arguments
    process_images(
        image1_path='image1.jpg',  # Path to first image (used for edge detection, etc.)
        image2_path='image2.jpg',  # Path to second image (used for noise removal)
        photo_path='photo.jpg'     # Path to color photo (used for color operations)
    )

Performing Sobel edge detection...
Performing histogram equalization...
Comparing smoothing filters...
Comparing edge detectors...
Applying morphological operations...
Separating color channels...
Isolating colors in HSV space...
Applying geometric transformations...
Adding and removing noise...

Completed 9 out of 9 operations successfully.
All image processing operations completed successfully.
Output images saved to current directory.


In [ ]:
# A grayscale image (image_histogram.jpg) has poor contrast and requires enhancment.


# 1.Write a Python Program to compute and plot the histogram of the image using Matplotlib.

# 2. Apply histogram equalization to enhance the image contrast and display the results.

# 3.Implement adaptive histogram equalization (CLAHE) and compare it with standart histogram equalization.
